In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
import urllib.request
import urllib.error
import time, itertools, re
from multiprocessing import Pool

# Setup Functions

In [3]:
# valHTML is a Python response from a get.
def findResultsPageMax(valHTML):
    rtnVal = -1
    valBS = BeautifulSoup(valHTML.text,'lxml')
    valTags = valBS.find_all('li')
    tmpURLs = []
    for i in valTags:
        valAs = i.find('a')
        try:
            tmpURLs.append(valAs.attrs['href'])
        except:
            None
    tmpAllURLs = pd.DataFrame({'URLs': tmpURLs})
    tmpSubset = tmpAllURLs.loc[tmpAllURLs['URLs'].str.contains('/sbirsearch/award/all\?page=')].copy()
    tmpSubset['Number'] = tmpSubset['URLs'].str.extract('([0-9]+)$',expand=False)
    tmpSubset['Number'] = pd.to_numeric(tmpSubset['Number'])
    tmpSubset.dropna(inplace=True)
    rtnVal = int(max(tmpSubset['Number']))
    return(rtnVal)

In [4]:
def getResultPage(valURL,valMaxTries=10):
    valCnt = 1
    isBad = True
    rtnVal = None
    while(isBad and valCnt < valMaxTries):
        try:
            tmpResponse = requests.get(valURL)
            if(tmpResponse.status_code==200):
                isBad = False
                rtnVal = tmpResponse
        except:
            None
        valCnt += 1
    return(rtnVal)

In [5]:
def getResultPageQuick(valURL):
    rtnValHTML = None
    rtnValStatusCode = -1
    try:
        tmpConnection = urllib.request.urlopen(valURL)
        rtnValStatusCode = tmpConnection.getcode()
        with tmpConnection as tmpResponse:
            rtnValHTML = tmpResponse.read()
    except urllib.error.HTTPError as e:
        print('HTTPError: {}'.format(e.code) + ', ' + url)
    return([rtnValStatusCode,valURL,rtnValHTML])

In [6]:
def getResultSBIRLinks(valHTML):
    valBS = BeautifulSoup(valHTML,'lxml')
    valTags = valBS.find_all('h3')
    tmpURLs = []
    for i in valTags:
        valAs = i.find('a')
        tmpURLs.append(valAs.attrs['href'])
    return(tmpURLs)

# Setup

In [7]:
valRetries = 10
valMin = 0
valURLBase = r'https://www.sbir.gov/sbirsearch/award/all?page='

valMax = findResultsPageMax(getResultPage(valURLBase+str(valMin),100))
valNumRun = 200
valRange = range(valMin,valNumRun)#valMax+1)

# Get SBIR Results
## Single Threaded

In [ ]:
%%time
tmpResultsPage = []
tmpSBIRLinks = pd.DataFrame()

for i in valRange:
    tmpURL = valURLBase+str(i)
    valSearchResultsPage = getResultPage(tmpURL,valRetries)
    tmpResultsPage.append(valSearchResultsPage)
    valSBIRLinks = getResultSBIRLinks(valSearchResultsPage.text)
    tmpDF = pd.DataFrame({'URLSBIR': valSBIRLinks})
    tmpDF['ResultsPage'] = i
    tmpSBIRLinks = tmpSBIRLinks.append(tmpDF)
    if((i % 1000)==0):
        print("Finished Retrieving "+str(i)+" Pages...")

In [ ]:
%%time
tmpSBIRLinks['URLSBIRev'] = r'https://www.sbir.gov'+tmpSBIRLinks['URLSBIR']
tmpSBIRLinks['SBIResponse'] = tmpSBIRLinks['URLSBIRev'].apply(lambda x: getResultPage(x,valRetries))
tmpSBIRLinks['HTMLText'] = tmpSBIRLinks['SBIResponse'].apply(lambda x: x.text)

In [ ]:
print(tmpSBIRLinks.shape)

In [ ]:
#tmpSBIRLinks.to_excel('./SBIRAwards.xlsx')

## Multi-Threaded

In [8]:
%%time
valRange = range(valMin,valNumRun)#valMax+1)
tmpURLs = [valURLBase+str(i) for i in list(valRange)]

if __name__ == "__main__":
    valProcessPool = Pool(processes=20)
    tmpResults = valProcessPool.map(getResultPageQuick, tmpURLs)
    tmpDFResults = pd.DataFrame(tmpResults,columns=['StatusCode','URL','Response'])
    valSBIRLinks = tmpDFResults.apply(lambda x: getResultSBIRLinks(x['Response']),axis=1)
    tmpDFSBIRLinks = pd.DataFrame(list(itertools.chain.from_iterable(valSBIRLinks)),columns=['URLSBIR'])

CPU times: user 23 s, sys: 422 ms, total: 23.4 s
Wall time: 1min 31s


In [9]:
%%time
if __name__ == "__main__":
    tmpDFSBIRLinks['URLSBIRev'] = r'https://www.sbir.gov'+tmpDFSBIRLinks['URLSBIR']
    tmpSBIRVals = valProcessPool.map(getResultPageQuick, tmpDFSBIRLinks['URLSBIRev'].tolist())
    tmpDFSBIRResults = pd.DataFrame(tmpSBIRVals,columns=['StatusCode','URL','HTMLText'])
    tmpSBIRResponse = pd.merge(left=tmpDFSBIRLinks,right=tmpDFSBIRResults,left_on='URLSBIRev',right_on='URL',how='outer')

CPU times: user 495 ms, sys: 252 ms, total: 747 ms
Wall time: 1min 30s


In [10]:
print(tmpDFResults.shape)
print(tmpSBIRResponse.shape)

(200, 3)
(2000, 5)


# Parse SBIR Records

In [11]:
def getSBIRAbstract(valSoup):
    #div class="abstract-wrapper"
    tmpFind = [i.text.strip() for i in valSoup.findAll("div", class_="abstract-wrapper")]
    tmpFind = ' '.join(tmpFind)
    tmpFind = re.sub(r'^Abstract','',tmpFind)
    tmpFind = tmpFind.strip()
    return(tmpFind)

def getSBIRDetails(valSoup):
    #span class="open-label"
    tmpFindLabels = [i.text for i in valSoup.findAll("span", class_="open-label")]
    tmpFindValues = [i.text for i in valSoup.findAll("span", class_="open-description")]
    return(tmpFindLabels,tmpFindValues)

In [12]:
%%time
tmpSBIRResponse2 = tmpSBIRResponse.copy()
tmpSBIRResponse2['Soup'] = tmpSBIRResponse2['HTMLText'].apply(lambda x: BeautifulSoup(x,'lxml'))
tmpSBIRResponse2['Abstract'] = tmpSBIRResponse2['Soup'].apply(getSBIRAbstract)
tmpOtherFields = tmpSBIRResponse2['Soup'].apply(getSBIRDetails)

tmpDFOtherField = pd.DataFrame()
for index,i in enumerate(tmpOtherFields):
    if(len(i[0])==len(i[1])):
        tmpValues = pd.DataFrame([i[1]],columns=i[0])
        tmpDFOtherField = tmpDFOtherField.append(tmpValues,ignore_index=True)
    else:
        print("Error at "+str(index))

tmpFinalSBIRs = tmpSBIRResponse2.join(tmpDFOtherField)
tmpFinalSBIRs = tmpFinalSBIRs.drop(['HTMLText','Soup','URLSBIR','URLSBIRev'],axis=1)

CPU times: user 1min 11s, sys: 1.17 s, total: 1min 12s
Wall time: 1min 12s


In [13]:
tmpWriter = pd.ExcelWriter(r'./SBIRAwards.xlsx',engine='xlsxwriter',options={'strings_to_urls': False})
tmpFinalSBIRs.to_excel(tmpWriter)
tmpWriter.close()